In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# 크롬 옵션 설정
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")

# 드라이버 실행
driver = webdriver.Chrome(options=options)
driver.get("https://map.naver.com/p/search/광진구%20키즈카페")

# 초기 iframe 진입
WebDriverWait(driver, 10).until(
    EC.frame_to_be_available_and_switch_to_it((By.ID, "searchIframe"))
)

results = []

for page in range(1, 50):  # 넉넉히 100페이지까지 시도
    print(f"[INFO] {page} 페이지 크롤링 중...")

    # 리스트 아이템 로딩 대기
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li._9v52G.UrAlx"))
        )
    except:
        print("❌ 리스트 로딩 실패, 중단")
        break

    # 아이템 수집
    items = driver.find_elements(By.CSS_SELECTOR, "li._9v52G.UrAlx")
    for item in items:
        try:
            name = item.find_element(By.CSS_SELECTOR, "span.CMy2_").text
        except:
            name = ""

        try:
            category = item.find_element(By.CSS_SELECTOR, "span.tEkkK").text
        except:
            category = ""

        try:
            address_full = item.find_element(By.CSS_SELECTOR, "span.o_gX8.Zdfo0").text
        except:
            address_full = ""

        print(f"{name} | {category} | {address_full}")
        results.append({
            "시설이름": name,
            "시설구분": category,
            "동이름": address_full,
        })

    # 다음 페이지 클릭
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, f"a[aria-label='{page+1}페이지']")
        driver.execute_script("arguments[0].click();", next_button)

        # 페이지 이동 후 iframe 다시 진입
        WebDriverWait(driver, 10).until(
            EC.frame_to_be_available_and_switch_to_it((By.ID, "searchIframe"))
        )

        time.sleep(1)  # 약간의 여유 대기
    except:
        print(f"[END] {page+1} 페이지 없음. 종료.")
        break

driver.quit()

# 결과 저장
df = pd.DataFrame(results)
df.to_csv("광진구_키즈카페_목록.csv", index=False, encoding="utf-8-sig")


[INFO] 1 페이지 크롤링 중...
캘리클럽 강변점 | 키즈카페,실내놀이터 | 서울 광진구 구의동
서울형키즈카페 시립뚝섬자벌레점 | 키즈카페,실내놀이터 | 서울 광진구 자양동
펀노리터 서울광진점 | 키즈카페,실내놀이터 | 서울 광진구 중곡동
데굴데굴키즈파티룸 구의점 | 키즈카페,실내놀이터 | 서울 광진구 구의동
퐁퐁플라워 건대센터 | 키즈카페,실내놀이터 | 서울 광진구 자양동
점핑몬스터 구의점 | 키즈카페,실내놀이터 | 서울 광진구 구의동
더키즈프리미엄키즈카페 | 키즈카페,실내놀이터 | 서울 광진구 중곡동
클래비키즈파크 | 키즈카페,실내놀이터 | 서울 광진구 능동
프로맘킨더 롯데백화점 건대스타시티점 | 키즈카페,실내놀이터 | 서울 광진구 자양동
서울형 키즈카페 광진구 중곡 3동점 | 키즈카페,실내놀이터 | 서울 광진구 중곡동
[END] 2 페이지 없음. 종료.
